In [126]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [127]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy, qiskit
import circuit_cutter
import mlrecon_methods as ml
import json

import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, H_distance_dict, total_counts
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [128]:
with open('HF_auckland_nocut_orign.json') as f:
    all_list = json.load(f)

In [129]:
len(all_list)

684

In [130]:
unmiti_list = all_list[-76:]
len(unmiti_list)

76

In [131]:
nocut_list = all_list[0:-76]
len(nocut_list)

608

In [132]:
# with open('BeH2_geneva_orign.json') as f:
#     unmiti_list = json.load(f)
with open('HF_auckland_check0.json') as f:
    miti_q0_list = json.load(f)
with open('HF_auckland_check1.json') as f:
    miti_q1_list = json.load(f)
with open('HF_auckland_check2.json') as f:
    miti_q2_list = json.load(f)
with open('HF_auckland_check3.json') as f:
    miti_q3_list = json.load(f)
with open('HF_auckland_check4.json') as f:
    miti_q4_list = json.load(f)
with open('HF_auckland_check5.json') as f:
    miti_q5_list = json.load(f)
with open('HF_auckland_check6.json') as f:
    miti_q6_list = json.load(f)
with open('HF_auckland_check7.json') as f:
    miti_q7_list = json.load(f)

In [133]:
len(nocut_list)

608

In [134]:
204/6

34.0

In [135]:
per_miti_results = []
for i in range(0, 8):
    temp_list = []
    for j in range(0, 76):
        filter_per_miti = norm_dict(filter_results(nocut_list[i * 76 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [136]:
per_miti_results[0]

[{'00000000': 0.006130491898992848,
  '00000001': 0.005400671434827032,
  '00010000': 0.005400671434827032,
  '00010001': 0.004232958692161728,
  '00010010': 0.005546635527660195,
  '00010011': 0.008028025105823967,
  '00010100': 0.003065245949496424,
  '00010101': 0.004524886877828055,
  '00010110': 0.003649102320829076,
  '00010111': 0.003211210042329587,
  '00011000': 0.004524886877828055,
  '00011001': 0.004524886877828055,
  '00011010': 0.003795066413662239,
  '00011011': 0.005108743249160707,
  '00011100': 0.003941030506495402,
  '00011101': 0.004086994599328565,
  '00011110': 0.0043789227849948915,
  '00011111': 0.00335717413516275,
  '00000010': 0.005984527806159685,
  '00100000': 0.004086994599328565,
  '00100001': 0.003211210042329587,
  '00100010': 0.003649102320829076,
  '00100011': 0.005838563713326522,
  '00100100': 0.00335717413516275,
  '00100101': 0.0021894613924974458,
  '00100110': 0.002627353670996935,
  '00100111': 0.0020434972996642826,
  '00101000': 0.00408699459

Untimigated distribution

In [137]:
qubits = 8
# list of all possible measurement outcomes (bitstrings)
all_bits = [ "".join(bits) for bits in itertools.product(["0","1"], repeat = qubits) ]

In [138]:
def load_probs(miti_dist_list, bit_str):
    output_list = []
    for dist in miti_dist_list:
        try:
            miti_count = dist[bit_str]
        except:
            miti_count = 0
        output_list.append(miti_count)
    return output_list

In [139]:
def bit_weight(dist, index):
    #bitwise distribution
    weight_0 = 0
    weight_1 = 0
    for key in dist.keys():
        if key[len(key) - 1 - index] == '0':
            weight_0 += dist[key]
        elif key[len(key) - 1 - index] == '1':
            weight_1 += dist[key]
        else:
            print("Incorrect key value")
    return weight_0, weight_1

In [140]:
def update_dist(unmiti_dist, miti_dist, index):
    Ppost = {}
    w0, w1 = bit_weight(miti_dist, index)
    u_w0, u_w1 = bit_weight(unmiti_dist, index)
    if w0 == 0:
        w0 = 0.0000000000001
        w1 = 0.9999999999999
    if w1 == 0:
        w1 = 0.0000000000001
        w0 = 0.9999999999999
    if u_w0 == 0:
        u_w0 = 0.0000000000001
        u_w1 = 0.9999999999999
    if u_w1 == 0:
        u_w1 = 0.0000000000001
        u_w0 = 0.9999999999999
        
    for key in unmiti_dist.keys():
        if key[len(key) - 1 - index] == '0':
            Ppost[key] = unmiti_dist[key] / u_w0 * (w0)# / w1)
            #print(w0, w1, w0/w1, Ppost[key])
        elif key[len(key) - 1 - index] == '1':
            Ppost[key] = unmiti_dist[key] / u_w1 * (w1)# / w0)
            #print(w0, w1, w1/w0, Ppost[key])
        else:
            print("Incorrect key value")
    return Ppost

In [141]:
def combine_dist(orign_dist, dist_list):
    output_dist = {}
    for key in orign_dist:
        value = orign_dist[key]
        for dist in dist_list:
            value += dist[key]
        output_dist[key] = value
    return output_dist  

In [142]:
def bayesian_reconstruct(unmiti_dist, miti_dist_list, threshold = 0.0001):
    temp_dist = unmiti_dist.copy()
    h_dist = 1
    while h_dist > threshold:
        temp_dist_start = temp_dist.copy()
        ppost = [0] * len(miti_dist_list)
        for i in range(0, len(miti_dist_list)):
            ppost[i] = update_dist(temp_dist, miti_dist_list[i][0], miti_dist_list[i][1])
        temp_dist = combine_dist(temp_dist, ppost)
        temp_dist = norm_dict(temp_dist)
        h_dist = H_distance_dict(temp_dist, temp_dist_start)
        print("H-dist:", h_dist)
    return temp_dist

In [143]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [162]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [per_miti_results[0][idx],0],[per_miti_results[1][idx],1], [per_miti_results[2][idx],2], [per_miti_results[3][idx],3], [per_miti_results[4][idx],4], [per_miti_results[5][idx],5], [per_miti_results[6][idx],6], [per_miti_results[7][idx],7]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133750215
H-dist: 0.0076653883848725275
H-dist: 0.006812706840258992
H-dist: 0.006056156497752209
H-dist: 0.0053843943810642225
H-dist: 0.00478759162773601
H-dist: 0.004257176537160592
H-dist: 0.003785635865309715
H-dist: 0.003366358087385786
H-dist: 0.0029935074296492626
H-dist: 0.00266192086205429
H-dist: 0.0023670225376942475
H-dist: 0.0021047517359388808
H-dist: 0.0018715014524755492
H-dist: 0.0016640655374280827
H-dist: 0.0014795928160365784
H-dist: 0.0013155470048203984
H-dist: 0.0011696715071495595
H-dist: 0.0010399583682275239
H-dist: 0.000924620813050615
H-dist: 0.000822068897482465
H-dist: 0.0007308878829501606
H-dist: 0.0006498190069476898
H-dist: 0.0005777423697701284
H-dist: 0.0005136616963662462
H-dist: 0.0004566907634669702
H-dist: 0.0004060413080334696
H-dist: 0.00036101225485405677
H-dist: 0.00032098011972034145
H-dist: 0.0002853904606857955
H-dist: 0.00025375026393169086
H-dist: 0.0002256211630971311
H-dist: 0.00020061340182984252
H-dist: 0.00017838045

H-dist: 0.002500223530230564
H-dist: 0.0022287565084149006
H-dist: 0.0019867832867730977
H-dist: 0.0017710942282671916
H-dist: 0.0015788313969004395
H-dist: 0.00140744902508083
H-dist: 0.0012546787839968437
H-dist: 0.001118499156963053
H-dist: 0.0009971083506815131
H-dist: 0.0008889002809256487
H-dist: 0.0007924432467573522
H-dist: 0.0007064609676761044
H-dist: 0.0006298157057520839
H-dist: 0.0005614932331223469
H-dist: 0.0005005894365728738
H-dist: 0.0004462983769746897
H-dist: 0.00039790164328963433
H-dist: 0.00035475885958582623
H-dist: 0.0003162992196507678
H-dist: 0.0002820139378310428
H-dist: 0.0002514495170264798
H-dist: 0.00022420174559741147
H-dist: 0.00019991034452064758
H-dist: 0.00017825419462781054
H-dist: 0.0001589470813155275
H-dist: 0.00014173390084801316
H-dist: 0.00012638727837572177
H-dist: 0.00011270455315070674
H-dist: 0.00010050509120445825
H-dist: 8.962789002611107e-05
H-dist: 70.00357133750926
H-dist: 0.008419307884626715
H-dist: 0.0074462466504347565
H-dist: 0.

H-dist: 0.003109667868964582
H-dist: 0.002783292618831966
H-dist: 0.0024906728238007626
H-dist: 0.002228357450020973
H-dist: 0.0019932547609931165
H-dist: 0.0017825905006134551
H-dist: 0.0015938723184837178
H-dist: 0.0014248592335733114
H-dist: 0.001273535190763191
H-dist: 0.0011380859676271015
H-dist: 0.0010168788484494502
H-dist: 0.0009084446079446103
H-dist: 0.0008114614452741245
H-dist: 0.000724740585173352
H-dist: 0.0006472133217168209
H-dist: 0.0005779193251239346
H-dist: 0.0005159960660570212
H-dist: 0.00046066923757332926
H-dist: 0.00041124407427144285
H-dist: 0.00036709748282259325
H-dist: 0.0003276709092457505
H-dist: 0.000292463876931352
H-dist: 0.00026102813625886553
H-dist: 0.0002329623722139554
H-dist: 0.0002079074210719353
H-dist: 0.00018554195123744605
H-dist: 0.00016557856690272408
H-dist: 0.00014776029642107931
H-dist: 0.00013185743027674217
H-dist: 0.00011766467630487115
H-dist: 0.00010499860241608117
H-dist: 9.36953395161316e-05
H-dist: 70.00357133750029
H-dist: 0.0

H-dist: 0.013754208665959774
H-dist: 0.012152664157366934
H-dist: 0.010754042376941466
H-dist: 0.009527385976666396
H-dist: 0.008448028035274113
H-dist: 0.007495903502855243
H-dist: 0.0066544063184334435
H-dist: 0.005909594555859002
H-dist: 0.005249623837060421
H-dist: 0.0046643341146377195
H-dist: 0.0041449417133388134
H-dist: 0.0036838050174791154
H-dist: 0.003274242624794859
H-dist: 0.00291038953640035
H-dist: 0.0025870814027923487
H-dist: 0.002299759828085988
H-dist: 0.002044393760730016
H-dist: 0.0018174133917382373
H-dist: 0.0016156539489657552
H-dist: 0.0014363074545085564
H-dist: 0.001276880992662665
H-dist: 0.0011351603791912128
H-dist: 0.0010091783705258352
H-dist: 0.0008971867325304132
H-dist: 0.0007976316222864486
H-dist: 0.000709131836717346
H-dist: 0.0006304595582816266
H-dist: 0.000560523287124751
H-dist: 0.0004983526956631791
H-dist: 0.0004430851789066714
H-dist: 0.0003939539042398825
H-dist: 0.0003502771895625649
H-dist: 0.00031144905982677276
H-dist: 0.000276930849978

H-dist: 0.006713661433532258
H-dist: 0.005980423654274873
H-dist: 0.005327858358460811
H-dist: 0.004746776465943089
H-dist: 0.00422915585172381
H-dist: 0.0037679538335449745
H-dist: 0.0033569605546133908
H-dist: 0.0029906821003665486
H-dist: 0.002664245587899993
H-dist: 0.002373320771844786
H-dist: 0.002114054290209867
H-dist: 0.0018830137657202888
H-dist: 0.0016771397385843434
H-dist: 0.00149370393955913
H-dist: 0.0013302727880586673
H-dist: 0.0011846752669363886
H-dist: 0.0010549745165504426
H-dist: 0.0009394426286668041
H-dist: 0.0008365382215527299
H-dist: 0.0007448864523071537
H-dist: 0.0006632611787558083
H-dist: 0.0005905690264623691
H-dist: 0.0005258351503024784
H-dist: 0.0004681905072091341
H-dist: 0.0004168604789369209
H-dist: 0.00037115470226830374
H-dist: 0.00033045797988141005
H-dist: 0.00029422215873658715
H-dist: 0.00026195887475890465
H-dist: 0.00023323307311191134
H-dist: 0.00020765722270768385
H-dist: 0.0001848861519495455
H-dist: 0.0001646124401921915
H-dist: 0.00014

H-dist: 0.00012041144927125642
H-dist: 0.00010814040306281897
H-dist: 9.712612578597887e-05
H-dist: 70.00357133798471
H-dist: 0.028610292341493638
H-dist: 0.02373042177864357
H-dist: 0.0203143535222302
H-dist: 0.0176851367638649
H-dist: 0.015557900939326862
H-dist: 0.01378428622634577
H-dist: 0.012276092223733225
H-dist: 0.010975934445450935
H-dist: 0.00984395732208755
H-dist: 0.008851077716224315
H-dist: 0.007975239720811579
H-dist: 0.007199195691917531
H-dist: 0.006509119862068434
H-dist: 0.005893703595428627
H-dist: 0.0053435429701591455
H-dist: 0.004850711050974234
H-dist: 0.00440845090387042
H-dist: 0.004010949906154379
H-dist: 0.0036531702078389755
H-dist: 0.003330718846519928
H-dist: 0.0030397464102859986
H-dist: 0.002776866604516496
H-dist: 0.002539091360373524
H-dist: 0.0023237776646855145
H-dist: 0.0021285833556916
H-dist: 0.0019514298778572701
H-dist: 0.0017904705227676245
H-dist: 0.0016440630671528482
H-dist: 0.0015107459967174603
H-dist: 0.0013892177053467399
H-dist: 0.001

H-dist: 0.0004417064716474016
H-dist: 0.00039440007062291983
H-dist: 0.00035211440694618925
H-dist: 0.00031432573000793974
H-dist: 0.0002805631771454635
H-dist: 0.00025040375514728477
H-dist: 0.00022346774172252444
H-dist: 0.00019941448080160488
H-dist: 0.0001779385459630603
H-dist: 0.00015876624676413172
H-dist: 0.00014165245335899548
H-dist: 0.00012637771553588215
H-dist: 0.00011274565319433786
H-dist: 0.0001005805962971429
H-dist: 8.97254534396561e-05
H-dist: 70.00357133751373
H-dist: 0.008884137951908486
H-dist: 0.007891902905560941
H-dist: 0.007011834786680951
H-dist: 0.006230634357385412
H-dist: 0.005536796215427981
H-dist: 0.004920301693606541
H-dist: 0.004372380841974928
H-dist: 0.0038853245339013106
H-dist: 0.0034523336022089725
H-dist: 0.003067395801918472
H-dist: 0.0027251840261608394
H-dist: 0.002420971011346757
H-dist: 0.0021505570325772378
H-dist: 0.0019102079869916608
H-dist: 0.001696601906222798
H-dist: 0.0015067824056746017
H-dist: 0.0013381179195953445
H-dist: 0.00118

H-dist: 0.00044013824802370056
H-dist: 0.00039519641574923544
H-dist: 0.000354896800198912
H-dist: 0.00031875109841384623
H-dist: 0.0002863239138988269
H-dist: 0.0002572267373923389
H-dist: 0.0002311126683742981
H-dist: 0.00020767177493254078
H-dist: 0.00018662700601953483
H-dist: 0.00016773058346763567
H-dist: 0.0001507608120517424
H-dist: 0.00013551925489601861
H-dist: 0.00012182822900560456
H-dist: 0.00010952858195921506
H-dist: 9.847771606050227e-05
H-dist: 70.00357133858199
H-dist: 0.03826579454765305
H-dist: 0.028163527360688004
H-dist: 0.022500013881942515
H-dist: 0.018683688465386625
H-dist: 0.015866776140694653
H-dist: 0.01367106950574116
H-dist: 0.01189700904688589
H-dist: 0.01042723074352232
H-dist: 0.00918718067873978
H-dist: 0.008126659255137841
H-dist: 0.007210274320706058
H-dist: 0.006412119432335099
H-dist: 0.005712628312018574
H-dist: 0.005096625648841499
H-dist: 0.0045520712152199464
H-dist: 0.0040692236678635954
H-dist: 0.0036400678583777234
H-dist: 0.003257912825762

H-dist: 0.00036934612070134025
H-dist: 0.0003416369081771201
H-dist: 0.0003159928137344998
H-dist: 0.00029226170741230304
H-dist: 0.00027030250350404
H-dist: 0.0002499843514154305
H-dist: 0.00023118589567284326
H-dist: 0.0002137945955692631
H-dist: 0.00019770609706522377
H-dist: 0.00018282365113594427
H-dist: 0.00016905757391557648
H-dist: 0.00015632474484654404
H-dist: 0.00014454813967441137
H-dist: 0.00013365639559935705
H-dist: 0.0001235834062479341
H-dist: 0.00011426794439551263
H-dist: 0.00010565331057674076
H-dist: 9.768700588334884e-05
H-dist: 70.00357133831913
H-dist: 0.03400575119009011
H-dist: 0.025360228739913532
H-dist: 0.020307533623680867
H-dist: 0.01686376809665329
H-dist: 0.014319448342459526
H-dist: 0.012343221083505853
H-dist: 0.010754985058489895
H-dist: 0.009446662696789034
H-dist: 0.008348699078473278
H-dist: 0.007413928151339385
H-dist: 0.006609030387767694
H-dist: 0.0059096722912070935
H-dist: 0.005297580856807859
H-dist: 0.0047587018539024094
H-dist: 0.004281997

H-dist: 0.0009731929036185073
H-dist: 0.0008968010591044756
H-dist: 0.0008268661623968595
H-dist: 0.000762770472852573
H-dist: 0.000703965811117522
H-dist: 0.0006499643776793488
H-dist: 0.0006003309332415395
H-dist: 0.0005546761235126726
H-dist: 0.0005126507676898546
H-dist: 0.00047394095995397396
H-dist: 0.0004382638580391918
H-dist: 0.0004053640534242517
H-dist: 0.00037501043471081704
H-dist: 0.0003469934699312746
H-dist: 0.00032112284536040957
H-dist: 0.00029722540829662587
H-dist: 0.0002751433695528738
H-dist: 0.00025473272832649956
H-dist: 0.00023586188792364945
H-dist: 0.00021841043567946233
H-dist: 0.0002022680644995656
H-dist: 0.00018733361687349996
H-dist: 0.00017351423508988695
H-dist: 0.00016072460380197695
H-dist: 0.0001488862731281811
H-dist: 0.00013792705218675346
H-dist: 0.0001277804644101153
H-dist: 0.00011838525720491847
H-dist: 0.00010968495955757117
H-dist: 0.00010162748205908501
H-dist: 9.416475456582257e-05
H-dist: 70.00357133853531
H-dist: 0.03827135682941523
H-di

In [163]:
len(full_dist)

76

In [164]:
pauli_list = read_from_file('HF_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [165]:
len(full_dist)

76

In [166]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [167]:
final_expect_val

(-27.12273117511683+0j)

In [168]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [miti_q0_list[idx],0],[miti_q1_list[idx],1], [miti_q2_list[idx],2], [miti_q3_list[idx],3], [miti_q4_list[idx],4], [miti_q5_list[idx],5], [miti_q6_list[idx],6],[miti_q7_list[idx],7]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133747441
H-dist: 0.003286375775429909
H-dist: 0.0029157046604714695
H-dist: 0.0025870750838865204
H-dist: 0.002295664909488854
H-dist: 0.0020372189074393293
H-dist: 0.0018079783554955059
H-dist: 0.0016046204184460419
H-dist: 0.001424205668391716
H-dist: 0.0012641324519729114
H-dist: 0.0011220970671972023
H-dist: 0.0009960589069124052
H-dist: 0.0008842098753892417
H-dist: 0.0007849475009950032
H-dist: 0.0006968512600904752
H-dist: 0.0006186617011683198
H-dist: 0.0005492620182612799
H-dist: 0.0004876617719793008
H-dist: 0.0004329824975318425
H-dist: 0.00038444497349934587
H-dist: 0.0003413579542438984
H-dist: 0.00030310819369205004
H-dist: 0.00026915160954533096
H-dist: 0.00023900545538030977
H-dist: 0.0002122413840567352
H-dist: 0.00018847929974285976
H-dist: 0.0001673819079957812
H-dist: 0.0001486498839531435
H-dist: 0.00013201758801072887
H-dist: 0.00011724926655005165
H-dist: 0.0001041356824909841
H-dist: 9.249112680036267e-05
H-dist: 70.0035713375579
H-dist: 0.01242

H-dist: 0.0003238761925981463
H-dist: 0.00028799646012500526
H-dist: 0.00025609988940911914
H-dist: 0.0002277434057838679
H-dist: 0.00020253331599922813
H-dist: 0.0001801197937817781
H-dist: 0.00016019198267022087
H-dist: 0.00014247364674507918
H-dist: 0.0001267193077441629
H-dist: 0.00011271081401427117
H-dist: 0.00010025429290238105
H-dist: 8.917744364095046e-05
H-dist: 70.00357133748786
H-dist: 0.005830662478423136
H-dist: 0.0051734496253949365
H-dist: 0.00459186916647299
H-dist: 0.004076759609368401
H-dist: 0.0036202052509674144
H-dist: 0.003215327539037028
H-dist: 0.002856121728156343
H-dist: 0.0025373266644572972
H-dist: 0.002254319256386661
H-dist: 0.00200302767939085
H-dist: 0.001779859053478804
H-dist: 0.0015816384961739808
H-dist: 0.001405557264027201
H-dist: 0.001249128267498634
H-dist: 0.0011101476519751476
H-dist: 0.0009866614324234675
H-dist: 0.000876936384854687
H-dist: 0.0007794345577283744
H-dist: 0.0006927908867403093
H-dist: 0.0006157934882762776
H-dist: 0.0005473662

H-dist: 0.0030934419037767615
H-dist: 0.002898862278000979
H-dist: 0.0027181148036403793
H-dist: 0.0025499990871637604
H-dist: 0.0023934443087494464
H-dist: 0.0022474929657128116
H-dist: 0.0021112867870472693
H-dist: 0.0019840545276671815
H-dist: 0.0018651013895963516
H-dist: 0.0017537998506564727
H-dist: 0.0016495817101001788
H-dist: 0.0015519311857909873
H-dist: 0.0014603789194869282
H-dist: 0.0013744967659577185
H-dist: 0.0012938932584057002
H-dist: 0.0012182096572609991
H-dist: 0.0011471165021379993
H-dist: 0.0010803105977964115
H-dist: 0.0010175123745451227
H-dist: 0.0009584635718383217
H-dist: 0.0009029252009995581
H-dist: 0.0008506757492114644
H-dist: 0.000801509592253493
H-dist: 0.0007552355880685098
H-dist: 0.0007116758271901693
H-dist: 0.0006706645194545049
H-dist: 0.0006320469993265287
H-dist: 0.0005956788346635738
H-dist: 0.0005614250258693406
H-dist: 0.0005291592842171708
H-dist: 0.0004987633796812981
H-dist: 0.00047012654994937366
H-dist: 0.00044314496343038335
H-dist: 0.

H-dist: 0.00147686316179493
H-dist: 0.0013923497753688197
H-dist: 0.0013126784566298891
H-dist: 0.0012375707569748273
H-dist: 0.001166764425721043
H-dist: 0.0011000124341663548
H-dist: 0.0010370820645578898
H-dist: 0.000977754058573451
H-dist: 0.0009218218205419534
H-dist: 0.0008690906711579817
H-dist: 0.0008193771478956988
H-dist: 0.0007725083487127968
H-dist: 0.0007283213159671485
H-dist: 0.0006866624577560534
H-dist: 0.0006473870041382008
H-dist: 0.0006103584959173477
H-dist: 0.0005754483038594972
H-dist: 0.0005425351763859159
H-dist: 0.0005115048139361239
H-dist: 0.00048224946833070036
H-dist: 0.0004546675655857327
H-dist: 0.00042866335074087615
H-dist: 0.00040414655336281903
H-dist: 0.00038103207247687943
H-dist: 0.0003592396797625259
H-dist: 0.0003386937399247954
H-dist: 0.0003193229472236358
H-dist: 0.00030106007720783823
H-dist: 0.000283841752759998
H-dist: 0.0002676082236143227
H-dist: 0.0002523031585605665
H-dist: 0.00023787344959524864
H-dist: 0.00022426902732591337
H-dist: 

H-dist: 0.0002000040784309861
H-dist: 0.00018855690105175303
H-dist: 0.00017776580941129485
H-dist: 0.00016759305373815553
H-dist: 0.000158003079960376
H-dist: 0.00014896239819141784
H-dist: 0.00014043945969805337
H-dist: 0.00013240454170975824
H-dist: 0.00012482963949087572
H-dist: 0.00011768836515102627
H-dist: 0.00011095585271686755
H-dist: 0.00010460866903040304
H-dist: 9.862473007632695e-05
H-dist: 70.00357133750956
H-dist: 0.0086717003126854
H-dist: 0.008108683260711593
H-dist: 0.0075884699216727325
H-dist: 0.007106974317829563
H-dist: 0.006660589323373686
H-dist: 0.0062461238731096776
H-dist: 0.005860748794739898
H-dist: 0.005501949990607061
H-dist: 0.005167487932633661
H-dist: 0.0048553626076563465
H-dist: 0.004563783181482503
H-dist: 0.004291141752980118
H-dist: 0.0040359906533556904
H-dist: 0.003797022816136045
H-dist: 0.003573054803779249
H-dist: 0.003363012129506192
H-dist: 0.003165916559272366
H-dist: 0.0029808751197187983
H-dist: 0.0028070705741078095
H-dist: 0.0026437531

...

H-dist: 0.00031620441487999684
H-dist: 0.00029811076368138953
H-dist: 0.0002810532666258205
H-dist: 0.00026497247376542016
H-dist: 0.0002498123618558127
H-dist: 0.00023552013467858593
H-dist: 0.00022204603529286202
H-dist: 0.00020934316946407633
H-dist: 0.0001973673395705566
H-dist: 0.00018607688833668766
H-dist: 0.0001754325517857626
H-dist: 0.00016539732084646294
H-dist: 0.00015593631108455186
H-dist: 0.00014701664006618278
H-dist: 0.00013860731189137192
H-dist: 0.00013067910846601056
H-dist: 0.00012320448710842767
H-dist: 0.00011615748411214221
H-dist: 0.00010951362391032117
H-dist: 0.00010324983350960415
H-dist: 9.734436188162584e-05
H-dist: 70.00357133755678
H-dist: 0.012675774115010145
H-dist: 0.01181311535424491
H-dist: 0.011022698366533502
H-dist: 0.010296502639727473
H-dist: 0.009627649616265051
H-dist: 0.009010210112220874
H-dist: 0.008439048767865515
H-dist: 0.007909697386950616
H-dist: 0.007418251077715108
H-dist: 0.006961282581083762
H-dist: 0.006535771239464567
H-dist: 0.

H-dist: 0.0044816873974413765
H-dist: 0.00420707421519126
H-dist: 0.0039510406631893535
H-dist: 0.0037120761682940167
H-dist: 0.003488825563667649
H-dist: 0.003280069732375102
H-dist: 0.003084708922412898
H-dist: 0.002901748345686648
H-dist: 0.0027302857335333337
H-dist: 0.002569500570407014
H-dist: 0.0024186447680381367
H-dist: 0.0022770345765970035
H-dist: 0.0021440435584396617
H-dist: 0.002019096474827672
H-dist: 0.0019016639572897094
H-dist: 0.001791257853575766
H-dist: 0.0016874271538762304
H-dist: 0.0015897544164970789
H-dist: 0.0014978526237991894
H-dist: 0.0014113624091824477
H-dist: 0.0013299496044470618
H-dist: 0.0012533030641889247
H-dist: 0.0011811327301516016
H-dist: 0.0011131679038121625
H-dist: 0.0010491557000496358
H-dist: 0.0009888596586444386
H-dist: 0.0009320584936817898
H-dist: 0.0008785449637647543
H-dist: 0.0008281248483554738
H-dist: 0.0007806160176183758
H-dist: 0.000735847584889292
H-dist: 0.0006936591323856923
H-dist: 0.000653900002043812
H-dist: 0.00061642864

H-dist: 0.011795029471608817
H-dist: 0.01085644590678607
H-dist: 0.010014695465854508
H-dist: 0.00925691625667041
H-dist: 0.008572342337008737
H-dist: 0.0079518879069004
H-dist: 0.0073878332321236405
H-dist: 0.006873582427442701
H-dist: 0.006403473144121111
H-dist: 0.005972624568884294
H-dist: 0.00557681430026842
H-dist: 0.005212377430562281
H-dist: 0.004876123023228775
H-dist: 0.004565264448192798
H-dist: 0.0042773609198524095
H-dist: 0.004010268204306158
H-dist: 0.0037620969078372085
H-dist: 0.003531177084198735
H-dist: 0.003316028140999873
H-dist: 0.003115333210444553
H-dist: 0.0029279172935962636
H-dist: 0.0027527286015742035
H-dist: 0.0025888226093645976
H-dist: 0.0024353484135903702
H-dist: 0.0022915370483592274
H-dist: 0.0021566914658735035
H-dist: 0.0020301779327877058
H-dist: 0.0019114186308071927
H-dist: 0.0017998852818673485
H-dist: 0.0016950936453130284
H-dist: 0.001596598757542169
H-dist: 0.0015039908041857557
H-dist: 0.0014168915315749984
H-dist: 0.0013349511184223802
H-d

H-dist: 0.0007675136309216222
H-dist: 0.0007236263292684481
H-dist: 0.000682247917568474
H-dist: 0.0006432350554046435
H-dist: 0.0006064525814480433
H-dist: 0.0005717730478523521
H-dist: 0.0005390762810790913
H-dist: 0.0005082489676469493
H-dist: 0.0004791842633886398
H-dist: 0.000451781424882342
H-dist: 0.0004259454618023325
H-dist: 0.00040158680900588187
H-dist: 0.00037862101724112714
H-dist: 0.00035696846142405856
H-dist: 0.0003365540654922863
H-dist: 0.00031730704289921073
H-dist: 0.00029916065186488453
H-dist: 0.00028205196454949205
H-dist: 0.00026592164936214095
H-dist: 0.0002507137656618076
H-dist: 0.00023637557014893676
H-dist: 0.00022285733428552667
H-dist: 0.00021011217211866362
H-dist: 0.0001980958779175641
H-dist: 0.000186766773067308
H-dist: 0.0001760855616937534
H-dist: 0.00016601519452368843
H-dist: 0.00015652074051220583
H-dist: 0.00014756926579567817
H-dist: 0.0001391297195536099
H-dist: 0.00013117282638620058
H-dist: 0.00012367098483665847
H-dist: 0.000116598171708302

H-dist: 0.0004494646165238311
H-dist: 0.00039988074409502965
H-dist: 0.00035578055726361637
H-dist: 0.00031655598969712573
H-dist: 0.0002816666096939841
H-dist: 0.00025063206472810404
H-dist: 0.0002230253763830427
H-dist: 0.00019846698845498215
H-dist: 0.000176619482515106
H-dist: 0.0001571828852626375
H-dist: 0.00013989050078511
H-dist: 0.00012450520855139638
H-dist: 0.00011081617474368867
H-dist: 9.863593050878957e-05
H-dist: 70.00357133757026
H-dist: 0.013586116644409457
H-dist: 0.012659828452922343
H-dist: 0.011812904085584976
H-dist: 0.011035875741207986
H-dist: 0.010320810141306173
H-dist: 0.009661000069903888
H-dist: 0.009050728490384534
H-dist: 0.008485085780370973
H-dist: 0.007959826440888941
H-dist: 0.007471255559207847
H-dist: 0.007016137992828072
H-dist: 0.006591625111779266
H-dist: 0.006195195256044839
H-dist: 0.005824605009776582
H-dist: 0.005477849079964717
H-dist: 0.0051531270720669405
H-dist: 0.004848815831455259
H-dist: 0.004563446303563541
H-dist: 0.00429568408242396

In [169]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [170]:
final_expect_val

(-28.030006398925543+0j)

In [50]:
final_expect_val

(-27.919626857102504+0j)

In [88]:
#no bayesian recombine:

In [112]:
miti_dist_list = [miti_q0_list, miti_q1_list, miti_q2_list, miti_q3_list, miti_q4_list, miti_q5_list, miti_q6_list, miti_q7_list]
total = 0
for idx in range(0, 8):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-27.86969089424107+0j)
expectation value for mitigating qubit1:(-27.857960514801597+0j)
expectation value for mitigating qubit2:(-27.804710022328436+0j)
expectation value for mitigating qubit3:(-27.7120384517776+0j)
expectation value for mitigating qubit4:(-27.767804963784066+0j)
expectation value for mitigating qubit5:(-27.776669962330736+0j)
expectation value for mitigating qubit6:(-27.753853735116977+0j)
expectation value for mitigating qubit7:(-27.744386596003224+0j)


In [113]:
#untimigated result:
unmiti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_expect_val += coeff * evaluation(unmiti_list[i], shots = total_counts(unmiti_list[i]), Pauli = Pauli_tuple[0])

In [114]:
unmiti_expect_val

(-27.58651417934895+0j)

In [122]:
bit_weight(full_dist[3], 0)

(0.48580962491518476, 0.5141903750848156)

In [124]:
bit_weight(miti_q0_list[3], 0)

(0.4858082362741587, 0.5141917637258404)

In [125]:
for i in range(len(full_dist)):
    print(bit_weight(full_dist[i], 0), bit_weight(miti_q0_list[i], 0))

(0.505353801727687, 0.4946461982723125) (0.505484303341783, 0.4945156966582177)
(0.5011337503677109, 0.49886624963228976) (0.5006356670863074, 0.4993643329136926)
(0.5075314455295895, 0.49246855447041027) (0.5073369318872052, 0.49266306811279525)
(0.48580962491518476, 0.5141903750848156) (0.4858082362741587, 0.5141917637258404)
(4.866958632414473e-06, 0.9999951330413678) (0, 1.0)
(0.4974454314132919, 0.5025545685867084) (0.49727100262712753, 0.5027289973728722)
(0.5017713272128252, 0.49822867278717425) (0.500793037154024, 0.49920696284597554)
(0.5006637739527947, 0.4993362260472054) (0.5006382383041346, 0.49936176169586555)
(0.49717773176272545, 0.5028222682372732) (0.49717898466403543, 0.5028210153359642)
(0.4981207826682764, 0.5018792173317231) (0.4981159100503956, 0.501884089949604)
(4.384917933276658e-06, 0.9999956150820657) (0, 0.9999999999999997)
(0.506635892770582, 0.4933641072294186) (0.5066268945929316, 0.4933731054070685)
(0.5003923538920709, 0.4996076461079292) (0.5003875907